In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import protfasta 
%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

In [3]:
Stanford = pd.read_excel("../data/Stanford_Supplementary Table 2 Domains from Tiles.xlsx")
Stanford = Stanford.rename(columns = {"UniProt ID" : "uniprotID"})
Stanford["Reference"] = "Stanford"
Stanford = Stanford[["HGNC symbol", "Start", "End", "uniprotID", "Reference"]]
Stanford

,HGNC symbol,Start,End,uniprotID,Reference
0,ABRAXAS1,121,200,Q6UWZ7,Stanford
1,AHR,531,640,P35869,Stanford
2,AHR,641,720,P35869,Stanford
3,AKAP8,1,80,O43823,Stanford
4,AKAP8L,1,80,Q9ULX6,Stanford
...,...,...,...,...,...
369,ZNF532,1,80,Q9HCE3,Stanford
370,ZNF592,1,80,Q92610,Stanford
371,ZNF644,901,980,Q9H582,Stanford
372,ZNF687,1,80,Q8N1G0,Stanford


In [4]:
Stanford_library_status = pd.read_csv("../data/minCMV ADs UPDATED 092023 with Ensembl full length.csv")
Stanford_library_status

,Unnamed: 0,HGNC symbol,Domain type,Domain,Start,End,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds
0,0,PPARG,AD,1,11,100,PESDSFTDTLSANISQEMTMVDTEMPFWPTNFGISSVDLSVMEDHS...,0.760083,TFTiles,90,ENST00000287820,ATGGGTGAAACTCTGGGAGATTCTCCTATTGACCCAGAAAGCGATT...
1,1,KAT6B,AD,1,1961,2060,MQRGMNMSVNLMPAPAYNVNSVNMNMNTLNAMNGYSMSQPMMNSGY...,-0.079427,CRTiles,100,ENST00000287239,ATGGTAAAACTTGCAAACCCACTTTATACAGAGTGGATTCTTGAAG...
2,2,NFE2L3,AD,1,251,340,HLNGTDTSFSLEDLFQLLSSQPENSLEGISLGDIPLPGSISDGMNS...,1.848783,TFTiles,90,ENST00000056233,ATGAAGCACCTGAAGCGGTGGTGGTCGGCCGGCGGCGGCCTCCTGC...
3,3,NFE2L3,AD,2,311,450,DVNLHEAILLCPNNTFRRDPTARTSQSQEPFLQLNSHTTNPEQTLP...,1.848783,TFTiles,140,ENST00000056233,ATGAAGCACCTGAAGCGGTGGTGGTCGGCCGGCGGCGGCCTCCTGC...
4,4,TP73,AD,1,11,90,GTTFEHLWSSLEPDSTYFDLPQSSRGNNEVVGGTDSSMDVFHLEGM...,0.116894,TFTiles,80,ENST00000378295,ATGGCCCAGTCCACCGCCACCTCCCCTGATGGGGGCACCACGTTTG...
...,...,...,...,...,...,...,...,...,...,...,...,...
369,369,NOBOX,AD,2,451,540,CGSWGTSITLPPPCSYLEELEPQDYQQSNQPGPFQFSQAPQPPLFQ...,0.935288,TFTiles,90,ENST00000483238,ATGGCTCTCCTTTTGACACTAACATCACCAGACCTGGAGGGTACCT...
370,370,NOBOX,AD,3,501,580,KLPYLPTFPFSMPSSLTLPPPEDSLFMFPCGPSGGTSQGYCPGASS...,0.935288,TFTiles,80,ENST00000483238,ATGGCTCTCCTTTTGACACTAACATCACCAGACCTGGAGGGTACCT...
371,371,NOBOX,AD,4,521,600,PEDSLFMFPCGPSGGTSQGYCPGASSGQILMQPPAGNIGTASWSDP...,0.935288,TFTiles,80,ENST00000483238,ATGGCTCTCCTTTTGACACTAACATCACCAGACCTGGAGGGTACCT...
372,372,ATF6,AD,1,1,90,GEPAGVAGTMESPFSPGLFHRLDEDWDSALFAELGYFTDTDELQLE...,1.274745,TFTiles,90,ENST00000367942,ATGGGGGAGCCGGCTGGGGTTGCCGGCACCATGGAGTCACCTTTTA...


In [5]:
# Adding library status
merged = pd.merge(Stanford, Stanford_library_status, on = ["HGNC symbol", "Start", "End"], how = "left")
merged

,HGNC symbol,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...
1,AHR,531,640,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...
2,AHR,641,720,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...
3,AKAP8,1,80,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...
4,AKAP8L,1,80,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,1,80,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...
370,ZNF592,1,80,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...
371,ZNF644,901,980,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...
372,ZNF687,1,80,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...


In [6]:
merged = merged.rename(columns = {"HGNC symbol" : "GeneName"})
merged

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...
1,AHR,531,640,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...
2,AHR,641,720,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...
3,AKAP8,1,80,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...
4,AKAP8L,1,80,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,1,80,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...
370,ZNF592,1,80,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...
371,ZNF644,901,980,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...
372,ZNF687,1,80,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...


In [7]:
# Adding 1 to Start and End if a transcription factor to make indexing consistent
merged["TF"] = merged["Sublibrary"] == "TFTiles"
merged["TF"] = merged["TF"].astype(int)
merged["Start"] += merged["TF"]
merged["End"] += merged["TF"]
merged

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1


In [8]:
all_isoform_seqs = protfasta.read_fasta("../data/uniprot_known_AD_seqs_isoforms.txt")
isoform_df = pd.DataFrame({"id":all_isoform_seqs.keys(),
             "seq":all_isoform_seqs.values()})
isoform_df["uniprotID"] = isoform_df["id"].str.split("|").str[1]
isoform_df

,id,seq,uniprotID
0,tr|A0A024R0Y4|A0A024R0Y4_HUMAN Transcriptional...,MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4
1,tr|A0A087WXG3|A0A087WXG3_HUMAN Basic helix-loo...,MSIRPPGEPPSPGGAAMAELKSLSGDAYLALSHGYAAAAAGLAYGA...,A0A087WXG3
2,sp|A6NJG6|ARGFX_HUMAN Arginine-fifty homeobox ...,MRNRMAPENPQPDPFINRNYSNMKVIPPQDPASPSFTLLSKLECSG...,A6NJG6
3,sp|A8MTJ6|FOXI3_HUMAN Forkhead box protein I3 ...,MALYCGDNFGVYSQPGLPPPAATAAAPGAPPAARAPYGLADYAAPP...,A8MTJ6
4,sp|A8MW92|P20L1_HUMAN PHD finger protein 20-li...,MSKKPPNRPGITFEIGARLEALDYLQKWYPSRIEKIDYEEGKMLVH...,A8MW92
...,...,...,...
1605,sp|Q9Y6Q9-5|NCOA3_HUMAN Isoform 5 of Nuclear r...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,Q9Y6Q9-5
1606,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1
1607,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
1608,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3


In [9]:
len("PTYPSLYSSYHQGCLVNPTGNLPMWSNQTWNNSTWSNQTQNIQSWSNHSWNTQTWCTQSWNNQAWNSPFYNCGEESLQSCMHFQPNSPASDLEAALEAAGEGLNVIQQTTRYFSTPQTMDLFLNYSMNMQPEDV")

134

In [10]:
305 - 162  + 1

144

In [11]:
AD_comparison_tools.return_uniprotID_isoform_mappings(merged, isoform_df)

# Questions for nicole:
# Seems like sequence of Q6NSW7 actually starts at 172 -- is sequence correct or are coordinates 
# Seems like sequence of P42229 actually starts 262 -- is sequence correct or are coordinates
# Seems like sequence of P86452 actually matches that of O75152 at the given coordinates

# Need to manually add A0A669KBM4 to isoform df

no matching isoforms for:
Q6NSW7
Tested:
1138    Q6NSW7
Name: uniprotID, dtype: object
162
305
expected:
Q6NSW7
PTYPSLYSSYHQGCLVNPTGNLPMWSNQTWNNSTWSNQTQNIQSWSNHSWNTQTWCTQSWNNQAWNSPFYNCGEESLQSCMHFQPNSPASDLEAALEAAGEGLNVIQQTTRYFSTPQTMDLFLNYSMNMQPEDV
observed:
['SNGVTQKASAPTYPSLYSSYHQGCLVNPTGNLPMWSNQTWNNSTWSNQTQNIQSWSNHSWNTQTWCTQSWNNQAWNSPFYNCGEESLQSCMHFQPNSPASDLEAALEAAGEGLNVIQQTTRYFSTPQTMDLFLNYSMNMQPEDV']

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
862
951
expected:
A0A669KBM4
KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPT
observed:
[]

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
1372
1460
expected:
A0A669KBM4
DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKDHPTVEMLG
observed:
[]

no matching isoforms for:
P42229
Tested:
631      P42229
632    P42229-2
Name: uniprotID, dtype: object
202
281
expected:
P42229
AGNGGPPEGSLDVLQSWCE

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,[Q6UWZ7]
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,[Q9ULX6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]"


In [12]:
262 + 281 - 202

341

In [13]:

# Questions for nicole:
# Seems like sequence of Q6NSW7 actually starts at 172 -- is sequence correct or are coordinates 
# Seems like sequence of P42229 actually starts 262 -- is sequence correct or are coordinates
# Seems like sequence of P86452 actually matches that of O75152 at the given coordinates

In [14]:
merged[merged["uniprotID"] == "Q6NSW7"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
201,NANOGP8,162,305,Q6NSW7,Stanford,311,AD,1,PTYPSLYSSYHQGCLVNPTGNLPMWSNQTWNNSTWSNQTQNIQSWS...,1.096882,TFTiles,134,ENST00000528386,ATGAGTGTGGATCCAGCTTGTCCCCAAAGCTTGCCTTGCTTTGAAG...,1,[]


In [15]:
merged[merged["uniprotID"] == "P42229"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
315,STAT5A,202,281,P42229,Stanford,197,AD,1,AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPG...,-1.668161,TFTiles,80,ENST00000345506,ATGGCGGGCTGGATCCAGGCCCAGCAGCTGCAGGGAGACGCGCTGC...,1,[]


In [16]:
merged[merged["uniprotID"] == "P86452"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
357,ZBED6,731,810,P86452,Stanford,55,AD,1,TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKL...,1.374137,TFTiles,80,ENST00000545588,ATGCCTAATCAAGGAGAAGACTGCTATTTTTTTTTCTATTCCACAT...,1,[]


In [17]:
merged[merged["uniprotID"] == "P86452"]["Sequence"].iloc[0]

'TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKLIWEISGGKLEAEIDLDPGKDEDDLLLELSEMIDS'

In [18]:
# The sequence for P86452 doesn't match the protein sequences given for the enst code
# https://useast.ensembl.org/Homo_sapiens/Transcript/Idhistory?t=ENST00000545588
# Also says that This transcript is not in the current gene set. Previous versions of the transcript may be available on the Transcript History page.

# The protein sequences given for the enst code do match the sequence for uniprotID O75152

In [19]:
merged[merged["uniprotID"] == "P42229"]["Sequence"].iloc[0]

'AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPGPVEEMLAEVNATITDIISALVTSTFIIEKQPPQV'

In [20]:
# After checking with Nicole 
# For ENST00000528386 it should start at 171 and end at 304.
# For ENST00000345506 it should start at 261 and end at 340.
# For ENST00000545588 (ZBED6), it aligns correctly to the very end of the cds, the annotated 730-809 is correct. 
    # SK: However, it seems as if that ENST ID is retired, and the seq actually matches that of O75152, will add a note to the final table
    

In [21]:
merged

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,[Q6UWZ7]
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,[Q9ULX6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]"


In [22]:
merged[merged["uniprotID"] == "Q6NSW7"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
201,NANOGP8,162,305,Q6NSW7,Stanford,311,AD,1,PTYPSLYSSYHQGCLVNPTGNLPMWSNQTWNNSTWSNQTQNIQSWS...,1.096882,TFTiles,134,ENST00000528386,ATGAGTGTGGATCCAGCTTGTCCCCAAAGCTTGCCTTGCTTTGAAG...,1,[]


In [23]:
merged.at[201, "Start"] = 172

In [24]:
merged[merged["uniprotID"] == "P42229"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
315,STAT5A,202,281,P42229,Stanford,197,AD,1,AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPG...,-1.668161,TFTiles,80,ENST00000345506,ATGGCGGGCTGGATCCAGGCCCAGCAGCTGCAGGGAGACGCGCTGC...,1,[]


In [25]:
merged.at[315, "Start"] = 262
merged.at[315, "End"] = 341

In [26]:
merged[merged["uniprotID"] == "P42229"]

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
315,STAT5A,262,341,P42229,Stanford,197,AD,1,AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPG...,-1.668161,TFTiles,80,ENST00000345506,ATGGCGGGCTGGATCCAGGCCCAGCAGCTGCAGGGAGACGCGCTGC...,1,[]


In [27]:
merged.at[357, "uniprotID"] = "O75152"

In [28]:
merged

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,[Q6UWZ7]
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,[Q9ULX6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]"


In [29]:
AD_comparison_tools.return_uniprotID_isoform_mappings(merged, isoform_df)

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
862
951
expected:
A0A669KBM4
KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPT
observed:
[]

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
1372
1460
expected:
A0A669KBM4
DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKDHPTVEMLG
observed:
[]

no matching isoforms for:
O75152
Tested:
Series([], Name: uniprotID, dtype: object)
731
810
expected:
O75152
TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKLIWEISGGKLEAEIDLDPGKDEDDLLLELSEMIDS
observed:
[]



,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,[Q6UWZ7]
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869]
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,[Q9ULX6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]"


In [30]:
isoform_df

,id,seq,uniprotID
0,tr|A0A024R0Y4|A0A024R0Y4_HUMAN Transcriptional...,MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4
1,tr|A0A087WXG3|A0A087WXG3_HUMAN Basic helix-loo...,MSIRPPGEPPSPGGAAMAELKSLSGDAYLALSHGYAAAAAGLAYGA...,A0A087WXG3
2,sp|A6NJG6|ARGFX_HUMAN Arginine-fifty homeobox ...,MRNRMAPENPQPDPFINRNYSNMKVIPPQDPASPSFTLLSKLECSG...,A6NJG6
3,sp|A8MTJ6|FOXI3_HUMAN Forkhead box protein I3 ...,MALYCGDNFGVYSQPGLPPPAATAAAPGAPPAARAPYGLADYAAPP...,A8MTJ6
4,sp|A8MW92|P20L1_HUMAN PHD finger protein 20-li...,MSKKPPNRPGITFEIGARLEALDYLQKWYPSRIEKIDYEEGKMLVH...,A8MW92
...,...,...,...
1605,sp|Q9Y6Q9-5|NCOA3_HUMAN Isoform 5 of Nuclear r...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,Q9Y6Q9-5
1606,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1
1607,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
1608,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3


In [31]:
# Adding the sequences of A0A669KBM4 and O75152 to isoform_df

# Uniprot says A0A669KBM4 is now obsolete
isoform_df.loc[len(isoform_df.index)] = ['>tr|A0A669KBM4|Release 2021_03/2021_03|02-Jun-2021',
                                         'MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQHKIKNSICKTVQSKVDCILQEVEKFTDLEKLYLYLQLPSGLSNGEKSDQNAMSSSRAQQMHAFSWIRNTLEEHPETSLPKQEVYDEYKSYCDNLGYHPLSAADFGKIMKNVFPNMKARRLGTRGKSKYCYSGLRKKAFVHMPTLPNLDFHKTGDGLEGAEPSGQLQNIDEEVISSACRLVCEWAQKVLSQPFDTVLELARFLVKSHYIGTKSMAALTVMAAAPAGMKGITQPSAFIPTAESNSFQPQVKTLPSPIDAKQQLQRKIQKKQQEQKLQSPLPGESAAKKSESATSNGVTNLPNGNPSILSPQPIGIVVAAVPSPIPVQRTRQLVTSPSPMSSSDGKVLPLNVQVVTQHMQSVKQAPKTPQNVPASPGGDRSARHRYPQILPKPANTSALTIRSPTTVLFTSSPIKTAVVPASHMSSLNVVKMTTISLTPSNSNTPLKHSASVSSATGTTEESRSVPQIKNGSVVSLQSPGSRSSSAGGTSAVEVKVEPETSSDEHPVQCQENSDEAKAPQTPSALLGQKSNTDGALQKPSNEGVIEIKATKVCDQRTKCKSRCNEMLPGTSTGNNQSTITLSVASQNLTFTSSSSPPNGDSINKDPKLCTKSPRKRLSSTLQETQVPPVKKPIVEQLSAATIEGQKQGSVKKDQKVPHSGKTEGSTAGAQIPSKVSVNVSSHIGANQPLNSSALVISDSALEQQTTPSSSPDIKVKLEGSVFLLDSDSKSVGSFNPNGWQQITKDSEFISASCEQQQDISVMTIPEHSDINDLEKSVWELEGMPQDTYSQQLHSQIQESSLNQIQAHSSDQLPLQSELKEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPTPTPTPTPTPTPTPTSEMIAGSQSLSRESPCSRLAQTTPVDSALGSSRHTPIGTPHSNCSSSVPPSPVECRNPFAFTPISSSMAYHDASIVSSSPVKPMQRPMATHPDKTKLEWMNNGYSGVGNSSVSGHGILPSYQELVEDRFRKPHAFAVPGQSYQSQSRHHDTHFGRLTPVSPVQHQGATVNNTNKQEGFAVPAPLDNKGTNSSASSNFRCRSVSPAVHRQRNLSGSTLYPVSNIPRSNVTPFGSPVTPEVHVFTNVHTDACANNIAQRSQSVPLTVMMQTAFPNALQKQANSKKITNVLLSKLDSDNDDAVRGLGMNNLPSNYTARMNLTQILEPSTVFPSANPQNMIDSSTSVYEFQTPSYLTKSNSTGQINFSPGDNQAQSEIGEQQLDFNSTVKDLLSGDSLQTNQQLVGQGASDLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKDHPTVEMLG',
                                         "A0A669KBM4"] 
# O75152
isoform_df.loc[len(isoform_df.index)] = ['>sp|O75152|ZC11A_HUMAN Zinc finger CCCH domain-containing protein 11A OS=Homo sapiens OX=9606 GN=ZC3H11A PE=1 SV=3',
                                         'MPNQGEDCYFFFYSTCTKGDSCPFRHCEAAIGNETVCTLWQEGRCFRQVCRFRHMEIDKKRSEIPCYWENQPTGCQKLNCAFHHNRGRYVDGLFLPPSKTVLPTVPESPEEEVKASQLSVQQNKLSVQSNPSPQLRSVMKVESSENVPSPTHPPVVINAADDDEDDDDQFSEEGDETKTPTLQPTPEVHNGLRVTSVRKPAVNIKQGECLNFGIKTLEEIKSKKMKEKSKKQGEGSSGVSSLLLHPEPVPGPEKENVRTVVRTVTLSTKQGEEPLVRLSLTERLGKRKFSAGGDSDPPLKRSLAQRLGKKVEAPETNIDKTPKKAQVSKSLKERLGMSADPDNEDATDKVNKVGEIHVKTLEEILLERASQKRGELQTKLKTEGPSKTDDSTSGARSSSTIRIKTFSEVLAEKKHRQQEAERQKSKKDTTCIKLKIDSEIKKTVVLPPIVASRGQSEEPAGKTKSMQEVHIKTLEEIKLEKALRVQQSSESSTSSPSQHEATPGARRLLRITKRTGMKEEKNLQEGNEVDSQSSIRTEAKEASGETTGVDITKIQVKRCETMREKHMQKQQEREKSVLTPLRGDVASCNTQVAEKPVLTAVPGITRHLTKRLPTKSSQKVEVETSGIGDSLLNVKCAAQTLEKRGKAKPKVNVKPSVVKVVSSPKLAPKRKAVEMHAAVIAAVKPLSSSSVLQEPPAKKAAVAVVPLVSEDKSVTVPEAENPRDSLVLPPTQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKLIWEISGGKLEAEIDLDPGKDEDDLLLELSEMIDS',
                                         "O75152"] 
isoform_df

,id,seq,uniprotID
0,tr|A0A024R0Y4|A0A024R0Y4_HUMAN Transcriptional...,MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4
1,tr|A0A087WXG3|A0A087WXG3_HUMAN Basic helix-loo...,MSIRPPGEPPSPGGAAMAELKSLSGDAYLALSHGYAAAAAGLAYGA...,A0A087WXG3
2,sp|A6NJG6|ARGFX_HUMAN Arginine-fifty homeobox ...,MRNRMAPENPQPDPFINRNYSNMKVIPPQDPASPSFTLLSKLECSG...,A6NJG6
3,sp|A8MTJ6|FOXI3_HUMAN Forkhead box protein I3 ...,MALYCGDNFGVYSQPGLPPPAATAAAPGAPPAARAPYGLADYAAPP...,A8MTJ6
4,sp|A8MW92|P20L1_HUMAN PHD finger protein 20-li...,MSKKPPNRPGITFEIGARLEALDYLQKWYPSRIEKIDYEEGKMLVH...,A8MW92
...,...,...,...
1607,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
1608,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3
1609,sp|Q9Y6Y1-4|CMTA1_HUMAN Isoform 4 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-4
1610,>tr|A0A669KBM4|Release 2021_03/2021_03|02-Jun-...,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,A0A669KBM4


In [84]:
Stanford_with_isoforms = AD_comparison_tools.return_uniprotID_isoform_mappings(merged, isoform_df)
Stanford_with_isoforms

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,148,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,[Q6UWZ7],1
1,AHR,532,641,P35869,Stanford,356,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869],1
2,AHR,642,721,P35869,Stanford,357,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,[P35869],1
3,AKAP8,2,81,O43823,Stanford,171,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,[O43823],1
4,AKAP8L,2,81,Q9ULX6,Stanford,115,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,[Q9ULX6],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,274,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,[Q9HCE3],1
370,ZNF592,2,81,Q92610,Stanford,62,AD,1,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,-0.700514,TFTiles,80,ENST00000560079,ATGGGGGATATGAAAACCCCAGATTTTGATGACCTTCTGGCTGCCT...,1,[Q92610],1
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2


---
Now, need to figure out which of the matching isoforms is the correct one.

Goal: if > 1 matching isoform, check each isoform's corresponding ENST code
If the corresponding ENST code matches the ENST code, then add isoform to list that is verified

If one of the matching isoforms is the canonical one, only add it to verified list if none of the other isoforms match ENST
This is because uniprot returns ENST codes associated with all isoforms of a uniprot ID if its canonical

Plan:
1. Filter merged to rows with > 1 matching isoform -> multip_matches
2. Download a list of all uniprot isoform IDs (non-canonical only)
3. Upload list to uniprot, use ID mapping to download mappings to ENST 
4. Iterate through multip_matches. For each list of matching isoforms, check to see if its ENST matches the ENST. If no matches, use canonical.

In [105]:
# 1. Filter merged to rows with > 1 matching isoform
Stanford_with_isoforms["matching_isoforms_count"] = [len(_) for _ in Stanford_with_isoforms["matching_isoforms"]]
multip_matches = Stanford_with_isoforms[Stanford_with_isoforms["matching_isoforms_count"] > 1]
multip_matches

,GeneName,Start,End,uniprotID,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count
5,APBB1,221,330,O00213,Stanford,91,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,CRTiles,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,"[O00213, O00213-2]",2
6,APBB3,11,90,O95704,Stanford,123,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6
7,APBB3,31,110,O95704,Stanford,124,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6
28,BRD2,471,550,P25440,Stanford,175,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,CRTiles,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,"[P25440, P25440-2]",2
29,BRD3,411,490,Q15059,Stanford,195,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,CRTiles,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,"[Q15059, Q15059-2]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,ZFHX4,1732,1811,Q86UP3,Stanford,291,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,TFTiles,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,"[Q86UP3-2, Q86UP3-5]",2
363,ZNF200,52,151,P98182,Stanford,102,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,"[P98182, P98182-3]",2
371,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2
372,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2


In [106]:
# 2. Download a list of all uniprot isoform IDs (non-canonical only)
import itertools
import csv

uniprotID_2d_list = list(multip_matches["matching_isoforms"])
uniprotID_list = list(itertools.chain.from_iterable(uniprotID_2d_list))
uniprotID_isoforms = list(filter(lambda uniprotID: "-" in uniprotID, uniprotID_list))

with open('../data/merged_uniprot_isoform_ids.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(uniprotID_isoforms)

uniprotID_isoforms

['O00213-2',
 'O95704-2',
 'O95704-3',
 'O95704-4',
 'O95704-5',
 'O95704-6',
 'O95704-2',
 'O95704-3',
 'O95704-4',
 'O95704-5',
 'O95704-6',
 'P25440-2',
 'Q15059-2',
 'O60885-2',
 'O60885-3',
 'Q6RI45-5',
 'Q9Y6Y1-2',
 'Q9Y6Y1-2',
 'O94983-2',
 'O94983-2',
 'O14647-2',
 'Q3L8U1-2',
 'Q3L8U1-3',
 'O96017-10',
 'O96017-12',
 'O96017-2',
 'O96017-5',
 'O96017-6',
 'O96017-7',
 'O96017-8',
 'O43889-3',
 'Q96BA8-2',
 'Q70SY1-2',
 'Q70SY1-3',
 'Q68CJ9-4',
 'Q68CJ9-5',
 'Q92793-2',
 'Q9P0U4-2',
 'Q9Y5R6-2',
 'P78545-2',
 'P03372-2',
 'P14921-2',
 'P14921-5',
 'P50549-3',
 'Q01780-2',
 'O00167-3',
 'O95677-3',
 'O95677-4',
 'Q12951-2',
 'Q6PIV2-2',
 'Q9NZI5-3',
 'Q8TE85-5',
 'Q8TE85-5',
 'O60381-2',
 'O60381-3',
 'P17509-2',
 'Q6IE81-3',
 'Q9NQC1-2',
 'Q9NQC1-3',
 'Q9NQC1-2',
 'Q9NQC1-3',
 'Q9H9V9-2',
 'O43474-1',
 'O75840-3',
 'Q8NEZ4-2',
 'Q8NEZ4-3',
 'Q8NEZ4-2',
 'Q8NEZ4-3',
 'O75030-2',
 'Q9HAP2-3',
 'Q9NP71-2',
 'Q9NP71-3',
 'Q9NP71-4',
 'Q9NP71-5',
 'P10242-10',
 'P10242-4',
 'P10242-

In [107]:
# 3. Upload list to uniprot, use ID mapping to download mappings to ENST 
ID_mappings = pd.read_csv("../data/merged_Stanford_isoform_mappings.txt", sep = "\t")
ID_mappings["ENST"] = ID_mappings["To"].str.split(".").str[0]
ID_mappings = ID_mappings.rename(columns = {"From" : "uniprotID"})
ID_mappings = ID_mappings[["uniprotID", "ENST"]]
ID_mappings

,uniprotID,ENST
0,O00213-2,ENST00000299402
1,O00213-2,ENST00000311051
2,O95704-2,ENST00000412920
3,O95704-3,ENST00000356738
4,O95704-4,ENST00000354402
...,...,...
122,Q9NPG3-2,ENST00000590769
123,Q86UP3-5,ENST00000651372
124,P98182-3,ENST00000396870
125,P98182-3,ENST00000396871


In [108]:
# 4. Merge ID_mappings with multip_matches
multip_matches_with_ENST_match = pd.merge(multip_matches, ID_mappings, left_on = "Canonical Transcript ID", right_on = "ENST", how = "left")
multip_matches_with_ENST_match

,GeneName,Start,End,uniprotID_x,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count,uniprotID_y,ENST
0,APBB1,221,330,O00213,Stanford,91,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,CRTiles,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,"[O00213, O00213-2]",2,O00213-2,ENST00000311051
1,APBB3,11,90,O95704,Stanford,123,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN
2,APBB3,31,110,O95704,Stanford,124,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN
3,BRD2,471,550,P25440,Stanford,175,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,CRTiles,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,"[P25440, P25440-2]",2,NaN,NaN
4,BRD3,411,490,Q15059,Stanford,195,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,CRTiles,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,"[Q15059, Q15059-2]",2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,ZFHX4,1732,1811,Q86UP3,Stanford,291,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,TFTiles,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,"[Q86UP3-2, Q86UP3-5]",2,Q86UP3-5,ENST00000651372
90,ZNF200,52,151,P98182,Stanford,102,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,"[P98182, P98182-3]",2,NaN,NaN
91,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2,NaN,NaN
92,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2,NaN,NaN


In [109]:
# All of the rows with matches are one of the isoforms in matching isoforms (doing what is expected!)
multip_matches_with_ENST_match[~multip_matches_with_ENST_match["uniprotID_y"].isna()]

,GeneName,Start,End,uniprotID_x,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count,uniprotID_y,ENST
0,APBB1,221,330,O00213,Stanford,91,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,CRTiles,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,"[O00213, O00213-2]",2,O00213-2,ENST00000311051
12,CHD9,1,80,Q3L8U1,Stanford,281,AD,1,MTDPMMDFFDDANLFGETLEGLSDDAFVQPGPVSLVDELNLGAEFE...,3.308836,CRTiles,80,ENST00000566029,ATGACAGATCCAATGATGGACTTTTTTGATGATGCCAATCTTTTTG...,0,"[Q3L8U1, Q3L8U1-2, Q3L8U1-3]",3,Q3L8U1-2,ENST00000566029
31,GRHL3,2,81,Q8TE85,Stanford,160,AD,1,SNELDFRSVRLLKNDPVNLQKFSYTSEDEAWKTYLENPLTAATKAM...,0.239785,TFTiles,80,ENST00000361548,ATGTCGAATGAACTTGATTTCAGGTCTGTGCGGCTGCTAAAGAACG...,1,"[Q8TE85, Q8TE85-5]",2,Q8TE85-5,ENST00000361548
32,GRHL3,22,101,Q8TE85,Stanford,161,AD,2,KFSYTSEDEAWKTYLENPLTAATKAMMRVNGDDDSVAALSFLYDYY...,0.239785,TFTiles,80,ENST00000361548,ATGTCGAATGAACTTGATTTCAGGTCTGTGCGGCTGCTAAAGAACG...,1,"[Q8TE85, Q8TE85-5]",2,Q8TE85-5,ENST00000361548
39,KLF4,42,121,O43474,Stanford,264,AD,1,NRWREELSHMKRLPPVLPGRPYDLAAATVATDLESGGAGAACGGSN...,0.096102,TFTiles,80,ENST00000374672,ATGAGGCAGCCACCTGGCGAGTCTGACATGGCTGTCAGCGACGCGC...,1,"[O43474, O43474-1]",2,O43474-1,ENST00000374672
49,MYT1L,152,251,Q9UL68,Stanford,12,AD,1,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...,0.739515,TFTiles,100,ENST00000399161,ATGGAGGTGGACACCGAGGAGAAGCGGCATCGCACGCGGTCCAAAG...,1,"[Q9UL68, Q9UL68-4]",2,Q9UL68-4,ENST00000399161
50,MYT1L,192,271,Q9UL68,Stanford,13,AD,2,DEYDNYDELVAKSLLNLGKIAEDAAYRARTESEMNSNTSNSLEDDS...,0.739515,TFTiles,80,ENST00000399161,ATGGAGGTGGACACCGAGGAGAAGCGGCATCGCACGCGGTCCAAAG...,1,"[Q9UL68, Q9UL68-4]",2,Q9UL68-4,ENST00000399161
51,MYT1L,312,391,Q9UL68,Stanford,14,AD,3,EKMVEESDEEVCLSSLECLRNQCFDLARKLSETNPQERNPQQNMNI...,0.739515,TFTiles,80,ENST00000399161,ATGGAGGTGGACACCGAGGAGAAGCGGCATCGCACGCGGTCCAAAG...,1,"[Q9UL68, Q9UL68-4]",2,Q9UL68-4,ENST00000399161
52,NCOA1,922,1011,Q15788,Stanford,81,AD,1,TVEGRNDEKALLEQLVSFLSGKDETELAELDRALGIDKLVQGGGLD...,2.939306,TFTiles,90,ENST00000288599,ATGAGTGGCCTCGGGGACAGTTCATCCGACCCTGCTAACCCAGACT...,1,"[Q15788, Q15788-2, Q15788-3]",3,Q15788-2,ENST00000288599
58,NR1H4,12,111,Q96RI1,Stanford,253,AD,1,HLPTTDEFSFSENLFGVLTEQVAGPLGQNLEVEPYSQYSNVQFPQV...,1.729773,TFTiles,100,ENST00000548884,ATGGGATCAAAAATGAATCTCATTGAACATTCCCATTTACCTACCA...,1,"[Q96RI1-1, Q96RI1-2, Q96RI1-5]",3,Q96RI1-2,ENST00000548884


In [110]:
# Assuming that the rows without matches match the canonical isoform
multip_matches_with_ENST_match[multip_matches_with_ENST_match["uniprotID_y"].isna()]

,GeneName,Start,End,uniprotID_x,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count,uniprotID_y,ENST
1,APBB3,11,90,O95704,Stanford,123,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN
2,APBB3,31,110,O95704,Stanford,124,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN
3,BRD2,471,550,P25440,Stanford,175,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,CRTiles,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,"[P25440, P25440-2]",2,NaN,NaN
4,BRD3,411,490,Q15059,Stanford,195,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,CRTiles,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,"[Q15059, Q15059-2]",2,NaN,NaN
5,BRD4,451,540,O60885,Stanford,96,AD,1,EMRFAKMPDEPEEPVVAVSSPAVPPPTKVVAPPSSSDSSSDSSSDS...,0.967550,CRTiles,90,ENST00000263377,ATGTCTGCGGAGAGCGGCCCTGGGACGAGATTGAGAAATCTGCCAG...,0,"[O60885, O60885-2, O60885-3]",3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,UBN1,241,370,Q9NPG3,Stanford,345,AD,1,GALSVKEMLKKFQKEKEAQKKREEEHKPVAVPSAEAQGLRELEGAS...,3.435097,CRTiles,130,ENST00000262376,ATGTCGGAGCCCCACAGGGTCCAGTTCACCTCTCTCCCAGGTTCCC...,0,"[Q9NPG3, Q9NPG3-2]",2,NaN,NaN
90,ZNF200,52,151,P98182,Stanford,102,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,"[P98182, P98182-3]",2,NaN,NaN
91,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2,NaN,NaN
92,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2,NaN,NaN


In [111]:
multip_matches_with_ENST_match["first_matching_isoform"] = [_[0] for _ in multip_matches_with_ENST_match["matching_isoforms"]]
multip_matches_with_ENST_match

,GeneName,Start,End,uniprotID_x,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count,uniprotID_y,ENST,first_matching_isoform
0,APBB1,221,330,O00213,Stanford,91,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,CRTiles,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,"[O00213, O00213-2]",2,O00213-2,ENST00000311051,O00213
1,APBB3,11,90,O95704,Stanford,123,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN,O95704
2,APBB3,31,110,O95704,Stanford,124,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN,O95704
3,BRD2,471,550,P25440,Stanford,175,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,CRTiles,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,"[P25440, P25440-2]",2,NaN,NaN,P25440
4,BRD3,411,490,Q15059,Stanford,195,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,CRTiles,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,"[Q15059, Q15059-2]",2,NaN,NaN,Q15059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,ZFHX4,1732,1811,Q86UP3,Stanford,291,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,TFTiles,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,"[Q86UP3-2, Q86UP3-5]",2,Q86UP3-5,ENST00000651372,Q86UP3-2
90,ZNF200,52,151,P98182,Stanford,102,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,"[P98182, P98182-3]",2,NaN,NaN,P98182
91,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2,NaN,NaN,Q9H582
92,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2,NaN,NaN,Q8N1G0


In [112]:
multip_matches_with_ENST_match['match'] = multip_matches_with_ENST_match['uniprotID_y'].fillna(multip_matches_with_ENST_match['first_matching_isoform'])
multip_matches_with_ENST_match

,GeneName,Start,End,uniprotID_x,Reference,Unnamed: 0,Domain type,Domain,Sequence,Max avg act,...,Length,Canonical Transcript ID,cds,TF,matching_isoforms,matching_isoforms_count,uniprotID_y,ENST,first_matching_isoform,match
0,APBB1,221,330,O00213,Stanford,91,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,...,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,"[O00213, O00213-2]",2,O00213-2,ENST00000311051,O00213,O00213-2
1,APBB3,11,90,O95704,Stanford,123,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,...,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN,O95704,O95704
2,APBB3,31,110,O95704,Stanford,124,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,...,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",6,NaN,NaN,O95704,O95704
3,BRD2,471,550,P25440,Stanford,175,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,...,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,"[P25440, P25440-2]",2,NaN,NaN,P25440,P25440
4,BRD3,411,490,Q15059,Stanford,195,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,...,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,"[Q15059, Q15059-2]",2,NaN,NaN,Q15059,Q15059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,ZFHX4,1732,1811,Q86UP3,Stanford,291,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,...,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,"[Q86UP3-2, Q86UP3-5]",2,Q86UP3-5,ENST00000651372,Q86UP3-2,Q86UP3-5
90,ZNF200,52,151,P98182,Stanford,102,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,...,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,"[P98182, P98182-3]",2,NaN,NaN,P98182,P98182
91,ZNF644,902,981,Q9H582,Stanford,263,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,...,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,"[Q9H582, Q9H582-2]",2,NaN,NaN,Q9H582,Q9H582
92,ZNF687,2,81,Q8N1G0,Stanford,32,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,...,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,"[Q8N1G0, Q8N1G0-2]",2,NaN,NaN,Q8N1G0,Q8N1G0


In [117]:
multip_matches_finalized = multip_matches_finalized.rename(columns = {"uniprotID_x" : "orig_uniprotID",
                                                                     "match" : "uniprotID_match"})

columns = ['GeneName', 'Start', 'End', 'orig_uniprotID', 'Reference', 
       'Domain type', 'Domain', 'Sequence', 'Max avg act', 'Sublibrary',
       'Length', 'Canonical Transcript ID', 'cds', 'TF',
       'uniprotID_match']

multip_matches_finalized = multip_matches_with_ENST_match[columns]

multip_matches_finalized

,GeneName,Start,End,orig_uniprotID,Reference,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,uniprotID_match
0,APBB1,221,330,O00213,Stanford,AD,1,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,1.178719,CRTiles,110,ENST00000311051,ATGTCTGTTCCATCATCACTGAGCCAGTCGGCCATTAATGCCAACA...,0,O00213-2
1,APBB3,11,90,O95704,Stanford,AD,1,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,O95704
2,APBB3,31,110,O95704,Stanford,AD,2,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,0.029374,CRTiles,80,ENST00000357560,ATGCTGGGCAAGGATTACATGCTGGCCATCATTCTGGTCAACTGCG...,0,O95704
3,BRD2,471,550,P25440,Stanford,AD,1,PGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEER...,-0.109781,CRTiles,80,ENST00000374825,ATGCTGCAAAACGTGACTCCCCACAATAAGCTCCCTGGGGAAGGGA...,0,P25440
4,BRD3,411,490,Q15059,Stanford,AD,1,RFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSE...,-0.739781,CRTiles,80,ENST00000303407,ATGTCCACCGCCACGACAGTCGCCCCCGCGGGGATCCCGGCGACCC...,0,Q15059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,ZFHX4,1732,1811,Q86UP3,Stanford,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,TFTiles,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,Q86UP3-5
90,ZNF200,52,151,P98182,Stanford,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,P98182
91,ZNF644,902,981,Q9H582,Stanford,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,Q9H582
92,ZNF687,2,81,Q8N1G0,Stanford,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,Q8N1G0


In [118]:
# Preparing the rest of Stanford_with_isoforms
single_matches = Stanford_with_isoforms[Stanford_with_isoforms["matching_isoforms_count"] == 1]
single_matches["uniprotID_match"] = [_[0] for _ in single_matches["matching_isoforms"]]
single_matches = single_matches.rename(columns = {"uniprotID" : "orig_uniprotID"})
single_matches = single_matches[columns]
single_matches

,GeneName,Start,End,orig_uniprotID,Reference,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,uniprotID_match
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,Q6UWZ7
1,AHR,532,641,P35869,Stanford,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,P35869
2,AHR,642,721,P35869,Stanford,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,P35869
3,AKAP8,2,81,O43823,Stanford,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,O43823
4,AKAP8L,2,81,Q9ULX6,Stanford,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,Q9ULX6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,ZNF469,832,911,H3BS19,Stanford,AD,2,LDMEDDAKLDSLITEALNGMEYQSDNPEIDSSFIDVFADEEPSGPR...,2.452388,TFTiles,80,ENST00000565624,ATGCCTGGGGAGCGCCCCCGAGGAGCGCCGCCCCCCACCATGACTG...,1,H3BS19
367,ZNF469,2892,2971,H3BS19,Stanford,AD,3,QPSFEEGGDPTLGPARLPTDLSDSSSLCLCHEDPWEDEDPAGLPES...,2.452388,TFTiles,80,ENST00000565624,ATGCCTGGGGAGCGCCCCCGAGGAGCGCCGCCCCCCACCATGACTG...,1,H3BS19
368,ZNF469,3002,3091,H3BS19,Stanford,AD,4,MPAPADDSSSSLGDVSPEPPSLERERCDGGLPGNTHLLPLRATDFE...,2.452388,TFTiles,90,ENST00000565624,ATGCCTGGGGAGCGCCCCCGAGGAGCGCCGCCCCCCACCATGACTG...,1,H3BS19
369,ZNF532,2,81,Q9HCE3,Stanford,AD,1,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,0.450331,TFTiles,80,ENST00000336078,ATGACCATGGGGGATATGAAGACCCCAGACTTTGATGACCTCCTGG...,1,Q9HCE3


In [121]:
Stanford_isoform_matches_finalized = pd.concat([single_matches, multip_matches_finalized])
Stanford_isoform_matches_finalized = Stanford_isoform_matches_finalized.reset_index(drop = True)
Stanford_isoform_matches_finalized

,GeneName,Start,End,orig_uniprotID,Reference,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,uniprotID_match
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,AD,1,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,-1.771253,CRTiles,80,ENST00000321945,ATGGAGGGGGAGAGTACGTCGGCGGTGCTCTCGGGCTTTGTGCTCG...,0,Q6UWZ7
1,AHR,532,641,P35869,Stanford,AD,1,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,1.159695,TFTiles,110,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,P35869
2,AHR,642,721,P35869,Stanford,AD,2,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,1.159695,TFTiles,80,ENST00000242057,ATGAACAGCAGCAGCGCCAACATCACCTACGCCAGTCGCAAGCGGC...,1,P35869
3,AKAP8,2,81,O43823,Stanford,AD,1,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,0.989791,TFTiles,80,ENST00000269701,ATGGACCAGGGCTACGGAGGCTACGGGGCGTGGAGTGCTGGACCTG...,1,O43823
4,AKAP8L,2,81,Q9ULX6,Stanford,AD,1,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,0.993831,TFTiles,80,ENST00000397410,ATGAGCTACACAGGCTTTGTCCAGGGATCTGAAACCACTTTGCAGT...,1,Q9ULX6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ZFHX4,1732,1811,Q86UP3,Stanford,AD,1,FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,-0.548176,TFTiles,80,ENST00000651372,ATGGAAACCTGTGACTCCCCTCCTATCTCAAGGCAGGAAAATGGGC...,1,Q86UP3-5
370,ZNF200,52,151,P98182,Stanford,AD,1,TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,-0.484594,TFTiles,100,ENST00000431561,ATGATGGCTGCAAAAGTGGTTCCTATGCCCCCAAAGCCAAAGCAGT...,1,P98182
371,ZNF644,902,981,Q9H582,Stanford,AD,1,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,-1.303710,TFTiles,80,ENST00000337393,ATGAGATCGTTCTTGCAGCAAGATGTTAATAAGACAAAATCTAGAC...,1,Q9H582
372,ZNF687,2,81,Q8N1G0,Stanford,AD,1,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,0.381333,TFTiles,80,ENST00000324048,ATGGGGGATATGAAGACCCCTGATTTTGATGACCTCCTTGCTGCCT...,1,Q8N1G0


In [126]:
# Will change orig_uniprotID to be correct here
Stanford_isoform_matches_finalized[Stanford_isoform_matches_finalized["uniprotID_match"] == "O75152"]

,GeneName,Start,End,orig_uniprotID,Reference,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,uniprotID_match
268,ZBED6,731,810,O75152,Stanford,AD,1,TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKL...,1.374137,TFTiles,80,ENST00000545588,ATGCCTAATCAAGGAGAAGACTGCTATTTTTTTTTCTATTCCACAT...,1,O75152


In [127]:
Stanford_isoform_matches_finalized.at[268, "orig_uniprotID"] = "P86452"
Stanford_isoform_matches_finalized[Stanford_isoform_matches_finalized["uniprotID_match"] == "O75152"]

,GeneName,Start,End,orig_uniprotID,Reference,Domain type,Domain,Sequence,Max avg act,Sublibrary,Length,Canonical Transcript ID,cds,TF,uniprotID_match
268,ZBED6,731,810,P86452,Stanford,AD,1,TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKL...,1.374137,TFTiles,80,ENST00000545588,ATGCCTAATCAAGGAGAAGACTGCTATTTTTTTTTCTATTCCACAT...,1,O75152


In [128]:
Stanford_isoform_matches_finalized.to_csv("../output/Stanford_isoform_matches_finalized.csv")